In [1]:
import pandas as pd
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
from decouple import Config, RepositoryEnv


In [2]:
# Load the environment variables using python-decouple
# The .env file should be in the root of the project
# The .env file should NOT be committed to the repository

config = Config(RepositoryEnv(".env.paul"))

# Load the credentials
WX_API_KEY = config("WX_API_KEY")

In [3]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)
client = APIClient(
    credentials=credentials, 
    project_id="68126b74-155e-4a70-aa2c-1781dfad87f6"
)

In [6]:
# --- Setup the Model (Granite-13b-chat-v2) ---
params = TextGenParameters(
    temperature=0.2,            # Light creativity
    max_new_tokens=300          # Enough for descriptive text
)

model = ModelInference(
    api_client=client,
    params=params,
    model_id="ibm/granite-13b-instruct-v2",
)
prompt = "Hello from copenhagen!"
generated_response = model.generate(prompt)

generated_response

{'model_id': 'ibm/granite-13b-instruct-v2',
 'created_at': '2025-04-28T20:23:19.759Z',
 'results': [{'generated_text': "I'm so excited to be here!",
   'generated_token_count': 9,
   'input_token_count': 6,
   'stop_reason': 'eos_token',
   'seed': 2640387947}]}

In [14]:
def create_prompt(sign_name: str) -> str:
    return (
        f"You are an American Sign Language (ASL) expert.\n\n"
        f"Please clearly explain how to perform the ASL sign for the concept '{sign_name}'. "
        f"Describe the hand shape, palm orientation, movement, location, and any facial expression needed."
    )


In [15]:
# --- Generate ASL Description ---
def generate_asl_description(sign_name: str) -> str:
    prompt = create_prompt(sign_name)
    
    response = model.generate(prompt=prompt)
    generated_text = response['results'][0]['generated_text']
    
    return generated_text

In [ ]:
# --- Example Usage ---
if __name__ == "__main__":
    sign_to_explain = input("📝 Enter the ASL sign you want explained: ")

    try:
        description = generate_asl_description(sign_to_explain)
        print("\n✅ Generated ASL Description:\n")
        print(description)
    except Exception as e:
        print(f"❌ Error during generation: {e}")

👋 Welcome to the ASL Sign Generator using Granite-13b!

✅ Generated ASL Description:

The sign for 'tree' is made by moving the index finger and thumb in a circular motion. The hand is held with the palm facing upwards. 


Judge

In [22]:
def create_judge_prompt(sign_name: str, description: str) -> str:
    return (
        f"You are an ASL (American Sign Language) expert evaluator.\n\n"
        f"Evaluate the following description for the ASL sign '{sign_name}':\n\n"
        f"---\n{description}\n---\n\n"
        f"Return ONLY a JSON object like this (no extra text):\n\n"
        f"{{\n"
        f"  \"accuracy\": <0 or 1>,\n"
        f"  \"fluffiness\": <0-10>\n"
        f"}}\n\n"
        f"IMPORTANT: Respond ONLY with the JSON. No explanations, no extra text."
    )



In [23]:
def evaluate_description(sign_name: str, description: str) -> dict:
    judge_prompt = create_judge_prompt(sign_name, description)

    # Use same model to judge
    response = model.generate(prompt=judge_prompt)
    judged_text = response['results'][0]['generated_text']

    try:
        # Safely parse JSON result
        import json
        judge_result = json.loads(judged_text)
    except Exception as e:
        print(f"❌ Could not parse judge response: {e}")
        judge_result = {"accuracy": None, "fluffiness": None}

    return judge_result


In [24]:
if __name__ == "__main__":
    print("👋 Welcome to the ASL Sign Generator using Granite-13b!")
    sign_to_explain = input("📝 Enter the ASL sign you want explained: ")

    try:
        # Generate ASL description
        description = generate_asl_description(sign_to_explain)
        print("\n✅ Generated ASL Description:\n")
        print(description)

        # Judge the description
        judge_result = evaluate_description(sign_to_explain, description)
        print("\n📊 Evaluation Result:\n")
        print(judge_result)

    except Exception as e:
        print(f"❌ Error during generation or evaluation: {e}")


👋 Welcome to the ASL Sign Generator using Granite-13b!

✅ Generated ASL Description:

Hello is signed by making a flat hand into a 'C' shape with the middle finger sticking up. The hand is held flat in front of the mouth with the palm facing forward. Hello is signed with the eyes looking forward and the mouth closed. 

📊 Evaluation Result:

{'accuracy': 0.9, 'fluffiness': 0}


Stop

In [9]:
models_to_test = {
    "Llama-3-8b": "meta-llama/llama-3-8b-instruct",
    "Granite-13b": "ibm/granite-13b-chat-v2",
    "Granite-20b": "ibm/granite-20b-multilingual",
}


In [ ]:

ZERO_SHOT_PROMPT = """You are an ASL (American Sign Language) instructor.

Your task is to describe how to perform the ASL sign for the following concept:

SIGN:
{sign_name}

Please provide a clear, simple, and beginner-friendly description.

Description:
"""

ONE_SHOT_PROMPT = """You are an expert ASL (American Sign Language) instructor.

Here is an example of a description:

EXAMPLE:
SIGN: Hello
Description: Raise your hand near your forehead, palm facing outward, and move your hand away from your forehead in a small arc.

Now, describe the following sign:

SIGN:
{sign_name}

Please provide a clear, step-by-step description.

Description:
"""

FEW_SHOT_PROMPT = """You are an expert ASL (American Sign Language) instructor.

Below are a few examples of how to describe ASL signs:

EXAMPLE 1:
SIGN: Hello
Description: Raise your hand near your forehead, palm facing outward, and move your hand away from your forehead in a small arc.

EXAMPLE 2:
SIGN: Thank You
Description: Place your fingertips near your chin and move your hand outward away from your face.

EXAMPLE 3:
SIGN: Apple
Description: Make a fist with your hand and twist it near your cheek.

Now, describe the following sign:

SIGN:
{sign_name}

Please provide a clear, step-by-step description.

Description:
"""

CHAIN_OF_THOUGHT_PROMPT = """You are an ASL (American Sign Language) instructor.

SIGN:
{sign_name}

Let's think step-by-step:
1. Identify the correct handshape needed for this sign.
2. Identify the correct palm orientation.
3. Identify the correct location relative to the body.
4. Identify the correct movement and motion path.
5. Mention if facial expressions are needed.

Explanation:
"""

CHAIN_OF_THOUGHT_APPEND = """
Based on the reasoning above, please now provide the full description:

Description:
"""


In [1]:
# 3. Prompting functions to insert sign_name dynamically

def zero_shot_prompt(sign_name):
    return ZERO_SHOT_PROMPT.format(sign_name=sign_name)

def one_shot_prompt(sign_name):
    return ONE_SHOT_PROMPT.format(sign_name=sign_name)

def few_shot_prompt(sign_name):
    return FEW_SHOT_PROMPT.format(sign_name=sign_name)

def chain_of_thought_prompt(sign_name):
    return CHAIN_OF_THOUGHT_PROMPT.format(sign_name=sign_name) + CHAIN_OF_THOUGHT_APPEND

# Mapping of style name to function
prompting_styles = {
    "Zero-Shot": zero_shot_prompt,
    "One-Shot": one_shot_prompt,
    "Few-Shot": few_shot_prompt,
    "Chain-of-Thought": chain_of_thought_prompt,
}


In [13]:
# 4. Define which models to test
models_to_test = {
    "Llama-3-8b": "meta-llama/llama-3-8b-instruct",
    "Granite-13b": "ibm/granite-13b-chat-v2",
    "Granite-20b": "ibm/granite-20b-multilingual",
}


In [2]:
# 5. Function to generate description using a model + prompt
def generate_with_model_and_prompt(sign_name, model_id, prompt_function):
    model = ModelInference(
        model_id=model_id,
        params=TextGenParameters(decoding_method="sample", max_new_tokens=500),
        client=client,
    )
    
    prompt = prompt_function(sign_name)
    
    response = model.generate(prompt=prompt)
    
    return response.get('results')[0]['generated_text']


In [17]:
# 6.A: Define the Judge Prompt Template

JUDGE_PROMPT_TEMPLATE = """You are an expert American Sign Language (ASL) instructor.

Evaluate the following generated description for the ASL sign '{sign_name}':

\"\"\" 
{generated_description}
\"\"\"

Assess it based on:
- Accuracy (correct description of handshape, movement, facial expression)
- Clarity (easy to understand for beginners)
- Completeness (no important details missing)

Provide:
1. A score from 1 to 10
2. A short justification.

Format your response like this:

Score: <number>/10
Justification: <your explanation>
"""



In [18]:
# 6.B: Judge function uses the template dynamically

def evaluate_with_judge(sign_name, generated_description):
    evaluation_prompt = JUDGE_PROMPT_TEMPLATE.format(
        sign_name=sign_name,
        generated_description=generated_description
    )
    
    response = judge_model.generate(prompt=evaluation_prompt)
    judged_text = response.get('results')[0]['generated_text']
    
    return judged_text


In [23]:
# 7. Run full comparison across models and prompts
def run_full_comparison_with_judging(sign_name):
    results = []

    for model_name, model_id in models_to_test.items():
        for prompt_name, prompt_func in prompting_styles.items():
            try:
                # Generate the description
                description = generate_with_model_and_prompt(sign_name, model_id, prompt_func)
                
                # Evaluate the description
                judge_feedback = evaluate_with_judge(sign_name, description)
                
                # Store all results
                results.append({
                    "Model": model_name,
                    "Prompt Style": prompt_name,
                    "Sign": sign_name,
                    "Description": description,
                    "Judge Feedback": judge_feedback
                })
            except Exception as e:
                results.append({
                    "Model": model_name,
                    "Prompt Style": prompt_name,
                    "Sign": sign_name,
                    "Description": f"Error: {e}",
                    "Judge Feedback": "N/A"
                })
    
    return pd.DataFrame(results)


In [ ]:
# 8. Input from user and run
if __name__ == "__main__":
    sign_to_test = input("Enter the ASL sign you want to explore: ")
    
    comparison_df = run_full_comparison_with_judging(sign_to_test)

    pd.set_option('display.max_colwidth', None)
    print(comparison_df)
